<a id="title"></a>
# WFC3/UVIS Time-dependent Photometry
***
## Learning Goals
By the end of this tutorial, you will: 
- Compute aperture photometry on FLC frames acquired at three unique epochs and apply the new time-dependent inverse sensitivity (PHOTFLAM) keywords.
- Recompute aperture photometry on modified FLC frames with 'equalized' countrate values.
- Redrizzle the 'equalized' FLC frames and compute aperture photometry on the DRC products.

## Table of Contents
[Introduction](#intro) <br>
[1. Imports](#imports) <br>
[2. Download the data](#data) <br>
[3. Correct for distortion using the Pixel Area Map](#pam) <br>

[4. Compute aperture photometry on the FLC frames](#phot) <br>
- [4.1 Calculate countrates](#calc_phot) <br>
- [4.2 Calculate magnitudes](#calc_mag) <br>
- [4.3 Plot countrate vs date](#plot_count) <br>
- [4.4 Plot magnitude vs date](#plot_mag) <br>

[5. Correct the FLC frames using 'photometric equalization'](#equal) <br>
- [5.1 Recompute countrates](#calc_equal) <br>
- [5.2 Plot corrected countrate vs date](#plot_equal) <br>

[6. Redrizzle the corrected FLC frames for each epoch](#astro) <br>
- [6.1 Recalculate photometry on the new DRC frames](#calc_astro) <br>
- [6.2 Plot DRC countrate vs date](#plot_astro) <br>

[7. Conclusions](#conclusions) <br>
[Additional Resources](#resources) <br>
[About the Notebook](#about) <br>
[Citations](#cite) <br>

<a id="intro"></a>
## Introduction

For UVIS images retrieved after October 15, 2020, new time-dependent photometry keyword values (PHOTFLAM, PHTFLAM1, PHTFLAM2 and PHTRATIO) are populated in the image header and must be applied separately for each observation epoch. This is a change from prior calibration, where a single set of keyword values were provided for each filter, independent of date. For more detail on the new calibration, see [WFC3 ISR 2021-04](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/wfc3/documentation/instrument-science-reports-isrs/_documents/2021/WFC3_ISR_2021-04.pdf).

In this tutorial, we show how to use the time-dependent calibration to compute aperture photometry on UVIS calibrated, CTE-corrected images (flc.fits, hereafter FLC) obtained at three epochs, spanning a total range of ~8 years and showing a loss in sensitivity of ~2%. The repository includes a CSV file containing a list of FLCs and the centroid of the star in each image, as well as the UVIS Pixel Area Maps to correct for distortion when working with FLC data.

Alternately, the FLC science arrays may be 'equalized' to account for sensitivity changes prior to computing photometry, where a reference set of keywords may be then used for all images. This photometric 'equalization' must be performed before combining any set of FLC images with AstroDrizzle which span multiple epochs in time.

<a id="imports"></a>
## 1. Imports

This notebook assumes you have created the virtual environment in [WFC3 Library's](https://github.com/spacetelescope/WFC3Library) installation instructions.

We import:
- *os* for setting environment variables
- *glob* for finding lists of files
- *shutil* for managing directories

- *numpy* for handling array functions
- *pandas* for managing data
- *matplotlib.pyplot* for plotting data
- *astroquery.mast Observations* for downloading data from MAST

- *astropy* for astronomy related functions
- *drizzlepac* for combining images
- *photutils* for photometric calculations
- *stwcs* for updating the World Coordinate System

In [ ]:
%matplotlib inline

import os
import glob
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy import wcs
from astroquery.mast import Observations

from drizzlepac import photeq
from drizzlepac import astrodrizzle

from photutils import aperture_photometry, CircularAperture, CircularAnnulus

from stwcs import updatewcs

<a id="data"></a>
## 2. Download the Data

The following commands query MAST for WFC3/UVIS calibrated (FLC) data products in the F606W filter for three epochs of GD153 observations (acquired in 2009, 2013, and 2017) and then downloads them to the current directory. 

In [ ]:
data_list = Observations.query_criteria(obs_id=['IBCDA4010','ICH3040F0','IDBHA6040'])
                                        
Observations.download_products(data_list['obsid'], project='CALWF3',
        mrp_only=False,download_dir='./data',productSubGroupDescription=['FLC','ASN'])

science_files = glob.glob('data/mastDownload/HST/*/*fits')

for im in science_files:
    root = im.split('/')[-1]
    os.rename(im,'./'+root)
shutil.rmtree('data/')

For convenience, the file 'GD153_F606W_public.csv' contains information about the images required for the tutorial. For each FLC image, the following is provided: filename, star x-cdt, y-cdt, filter, CCD amplifier, chip, and epoch for the 3 observation dates.

In [ ]:
df = pd.read_csv('GD153_F606W_public.csv')

df

<a id="pam"></a>
## 3. Correct for distortion using the Pixel Area Map

FLC frames are not corrected for distortion and pixels therefore do not have equal area on the sky. To correct for this affect, we multiply the FLC frames by the [Pixel Area Map (PAM)](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/pixel-area-maps). Since the GD153 data are C512C subarrays, the PAM needs to be "cut out" at the region corresponding to the subarray.

In [ ]:
pam1 = fits.getdata('UVIS1wfc3_map.fits')
pam2 = fits.getdata('UVIS2wfc3_map.fits')

pams = {'A': pam1[-512:, :513], 'B': pam1[-512:, -513:], 'C': pam2[:512, :513], 'D': pam2[:512, -513:]}

<a id="phot"></a>
## 4. Compute aperture photometry on the FLC frames

<a id="calc_phot"></a>
### 4.1 Calculate countrates

Here we set up the code that computes the photometry. We are using a standard aperture size of 10 pixels, with a sky annulus from 155 to 165 pixels. These are the steps involved in computing the photometry:

1. Loop through each FLC and load the data, MJD date of exposure, exposure time, and PHOTFLAM.
2. Divide each FLC by the exposure time to get countrates in electrons per second (FLC images are in electrons).
3. Correct for distortion by multiplying by the appropriate cutout of the pixel area map (PAM) corresponding to the subarray region.
4. Supply the x,y coordinate of the star in each image from the CSV file/pandas dataframe.
5. Define the aperture, annulus aperture, and annulus mask.
6. Compute the sigma-clipped mean of the sky annulus.
7. Compute the photometry in the aperture.
8. Subtract the sky background from the photometry derived in the previous step.
9. Store all values.

In [ ]:
phots = []
mjds = []
dates = []
ap = 10
skyrad = [155, 165]
pfl = []
for i, flc in enumerate(df['FLC'].values):
    with fits.open(flc) as f:
        data = f[1].data
        mjd = f[0].header['EXPSTART']
        date = f[0].header['DATE-OBS']
        exptime = f[0].header['EXPTIME']
        pfl.append(f[0].header['PHOTFLAM'])
    data = data / exptime
    data = data * pams[df.at[i, 'Amp']]

    positions = (df.at[i, 'Centx'], df.at[i, 'Centy'])
    aperture = CircularAperture(positions, ap)
    annulus_aperture = CircularAnnulus(positions, r_in=skyrad[0], r_out=skyrad[1])
    annulus_masks = annulus_aperture.to_mask(method='center')
    annulus_data = annulus_masks.multiply(data)
    mask = annulus_masks.data
    annulus_data_1d = annulus_data[mask > 0]
    mean_sigclip, _, _ = sigma_clipped_stats(annulus_data_1d)
    
    apers = [aperture, annulus_aperture]
    phot_table = aperture_photometry(data,apers)

    background = mean_sigclip * aperture.area
    final_sum = phot_table['aperture_sum_0'] - background
    phots.append(final_sum[0])
    mjds.append(mjd)    
    dates.append(date)

Next, we append the coutrates, MJD's, and PHOTFLAM values to our dataframe.

In [ ]:
df['Countrate'] = phots
df['MJD'] = mjds
df['DATE-OBS'] = dates
df['PHOTFLAM'] = pfl

df

<a id="calc_mag"></a>
### 4.2 Calculate magnitudes

We now convert countrates into ST magnitudes using the PHOTFLAM value and the following equation. The $EE_{r10}$ is the encircled energy term for an aperture radius of r=10 pixels (0.4 arcseconds). For F606W with UVIS2, this is 0.91. This value can be computed using `stsynphot` as described in the 'Photometry Examples' notebook in this WFC3 Library repository.

In [ ]:
EE_r10 = 0.91
df['STMags'] = -21.1 -2.5*np.log10(df['PHOTFLAM']) -2.5*np.log10(df['Countrate']) - 2.5 * np.log10(1./EE_r10)

In [ ]:
df

<a id="plot_count"></a>
### 4.3 Plot countrate vs date

We first plot the photometric countrates (electrons per second) vs time in MJD. The decline in the observed countrate is due to sensitivity loss in F606W at a rate of ~0.2% per year.

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=40)

plt.plot(df['MJD'], df['Countrate'], 'o', markersize=20, label='Amp C')
plt.grid()
plt.xlabel('MJD', fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.ylabel(r'Countrate (e$^-$/s)', fontsize=30)
plt.title('FLC Countrates for GD153 in F606W', fontsize=30)
plt.ylim(101000, 104000)
plt.legend(loc=0, fontsize=30)

<a id="plot_mag"></a>
### 4.4 Plot magnitude vs date

Now, we plot the ST magnitude versus time in MJD. This is computed using the 'corrected' PHOTFLAM keyword, so the magnitude values are stable over time.

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=40)

plt.plot(df['MJD'], df['STMags'], 'o', markersize=20, label='Amp C')
plt.grid()
plt.ticklabel_format(useOffset=False)
plt.xlabel('MJD', fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.ylabel('STMAG (Magnitude)', fontsize=30)
plt.ylim(13.55, 13.65)
plt.title('STMAG for GD153 in F606W on UVIS2', fontsize=30)
plt.legend(loc=0, fontsize=30)

<a id="equal"></a>
## 5. Correct the FLC frames using 'photometric equalization'

This single-line step will equalize the countrates in the science array of the FLC frames to match any specified 'reference' image. For more details, see the drizzlepac documentation for the [phot_eq software](https://drizzlepac.readthedocs.io/en/latest/photeq.html). Note that at this step, we overwrite the science pixels in the original FLC files. In this case the data are sorted, and the software automatically uses the PHOTFLAM value of the first 2009 image as a reference for matching the with the other images. You can supply a given reference PHOTFLAM value to the photeq call and/or ensure that your data are time-sorted.

**Note: Running this cell will edit the FLCs in the local directory, and you will need to download the files again if you require the original data.**

In [ ]:
photeq.photeq(','.join(df['FLC'].values), readonly=False)

<a id="calc_equal"></a>
### 5.1 Recompute countrates

We repeat the same aperture photometry as in [Section 4.1](#calc_phot) but using the photometrically equalized FLC data.

In [ ]:
phots = []
for i, flc in enumerate(df['FLC'].values):
    with fits.open(flc) as f:
        data = f[1].data
        exptime = f[0].header['EXPTIME']
    data = data / exptime
    data = data * pams[df.at[i, 'Amp']]

    positions = (df.at[i, 'Centx'], df.at[i, 'Centy'])
    aperture = CircularAperture(positions, ap)
    annulus_aperture = CircularAnnulus(positions, r_in=skyrad[0], r_out=skyrad[1])
    annulus_masks = annulus_aperture.to_mask(method='center')
    annulus_data = annulus_masks.multiply(data)
    mask = annulus_masks.data
    annulus_data_1d = annulus_data[mask > 0]
    mean_sigclip, _, _ = sigma_clipped_stats(annulus_data_1d)
    background = mean_sigclip * aperture.area
    
    apers = [aperture, annulus_aperture]
    phot_table = aperture_photometry(data,apers)

    final_sum = phot_table['aperture_sum_0'] - background
    phots.append(final_sum[0])
    mjds.append(mjd)

We'll append the photometrically equalized countrates to our dataframe.

In [ ]:
df['Phot-eq'] = phots

In [ ]:
df

<a id="plot_equal"></a>
### 5.2 Plot corrected countrate vs date

We plot the photometric countrate (electrons per second) versus time in MJD. The corrected data in blue shows that the countrate is now ~flat over time to within the measurment errors. The original countrates are shown in grey and show a decline of nearly 2% over the three epochs.

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=40)

plt.plot(df['MJD'], df['Countrate'], 's', markersize=20, label='Not equalized', alpha=0.55, color='Grey')
plt.plot(df['MJD'], df['Phot-eq'], 'o', markersize=18, label='Equalized')
plt.grid()
plt.xlabel('MJD', fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.ylabel('Countrate (e-/s)', fontsize=30)
plt.title('Original and Corrected countrates for GD153 in F606W', fontsize=30)
plt.ylim(101000, 104000)
plt.legend(loc=0, fontsize=30)

<a id="astro"></a>
## 6. Redrizzle the corrected FLC frames for each epoch

The corrected FLC data from each epoch can now be redrizzled to correct for distortion, to remove cosmic rays and bad pixels, and to improve the signal-to-noise in the combined DRC product at each date. (Alternately, the entire set of FLC images may be combined to produce a single DRC image to use for photometry.)

**Warning: This cell may take a few minutes to complete.**

In [ ]:
for asn in glob.glob('*asn.fits'):
    astrodrizzle.AstroDrizzle(asn,
                skymethod= 'match',
                skystat='mean',
                driz_sep_bits='80',
                combine_type='median',
                combine_nhigh=1,
                driz_cr_snr= '3.5 3.0', 
                driz_cr_scale= '2.0 1.5',
                final_bits= '80', 
                build=True,
                clean=True,
                preserve=False,
                num_cores=1)

In [ ]:
drcs = ['ibcda4010_drc.fits', 'ich3040f0_drc.fits', 'idbha6040_drc.fits']
drcents = [(327.9, 341.4), (251.1, 350.0), (266.6, 273.6)]

<a id="calc_astro"></a>
### 6.1 Recalculate photometry on the new DRC frames

We now perform the photometry on the drizzled (DRC) products, using similar techniques as in [Section 4.1](#calc_phot). Drizzled images are in units of electrons per second are already corrected for distortion, so we no longer need to divide by the exposure time or apply the PAM prior to computing photometry.

In [ ]:
phots = []
mjds = []
pfls = []
for i, drc in enumerate(drcs):
    data = fits.getdata(drc, ext=1)
    
    mjds.append(np.mean(df.query('Epoch == {}'.format(i+1))['MJD']))
    pfls.append(df.query('Epoch == {}'.format(i+1))['PHOTFLAM'].values[0])

    positions = drcents[i]
    aperture = CircularAperture(positions, ap)
    annulus_aperture = CircularAnnulus(positions, r_in=skyrad[0], r_out=skyrad[1])
    annulus_masks = annulus_aperture.to_mask(method='center')
    annulus_data = annulus_masks.multiply(data)
    mask = annulus_masks.data
    annulus_data_1d = annulus_data[mask > 0]
    mean_sigclip, _, _ = sigma_clipped_stats(annulus_data_1d)
    background = mean_sigclip * aperture.area
    
    apers = [aperture, annulus_aperture]
    phot_table = aperture_photometry(data,apers)

    final_sum = phot_table['aperture_sum_0'] - background
    phots.append(final_sum[0])

In [ ]:
mags = -21.1 - 2.5*np.log10(pfls[0]) - 2.5*np.log10(phots) - 2.5*np.log10(1./EE_r10)
print(mags)

<a id="plot_astro"></a>
### 6.2 Plot DRC countrate vs date

The corrected photometric countrate from the DRC and the recomputed magnitudes are plotted below. Note how they agree with the results from the previous step using the FLC and the PAM.

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=40)

plt.plot(df['MJD'], df['Phot-eq'], 'd', markersize=20, color='grey', label='FLC photometrically equalized', alpha=0.5)
plt.plot(mjds, phots, 'o', markersize=20, label='DRC photometry')
plt.grid()
plt.xlabel('MJD', fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.ylabel('Countrate (e-/s)', fontsize=30)
plt.title('GD153, F606W, UVIS2, DRC', fontsize=30)
plt.ylim(101000, 104000)
plt.legend(loc=4, fontsize=30)

In [ ]:
fig = plt.figure(figsize=(20, 10), dpi=40)

plt.plot(df['MJD'], df['STMags'], 's', markersize=25, label='FLC', alpha=0.4, color='Grey')
plt.plot(mjds, mags, 'o', markersize=20, label='DRC')
plt.grid()
plt.ticklabel_format(useOffset=False)
plt.xlabel('MJD', fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.ylabel('STMAG (Magnitude)', fontsize=30)
plt.ylim(13.55, 13.65)
plt.title('GD153, F606W, UVIS2, DRC', fontsize=30)
plt.legend(loc=4, fontsize=30)

<a id="conclusions"></a>
## 7. Conclusions

Thank you for walking through this notebook. Now using WFC3 data, you should be more familiar with:

- Computing aperture photometry and magnitudes on:
    - FLC frames using new time-dependent photometry keywords.
    - FLC frames with equalized countrate values.
    - DRC frames produced from corrected FLCs.

#### Congratulations, you have completed the notebook!

<a id="resources"></a>
## Additional Resources
Below are some additional resources that may be helpful. Please send any questions through the [HST Helpdesk](https://stsci.service-now.com/hst).

- [WFC3 Website](https://www.stsci.edu/hst/instrumentation/wfc3)
- [WFC3 Instrument Handbook](https://hst-docs.stsci.edu/wfc3ihb)
- [WFC3 Data Handbook](https://hst-docs.stsci.edu/wfc3dhb)
    - see section 9.5.2 for reference to this notebook
    
<a id="about"></a>
## About this Notebook

**Authors:** Harish Khandrika, Jennifer Mack; WFC3 Instrument Team

**Updated on:** 2021-09-10

<a id="cite"></a>
## Citations

If you use `numpy`, `astropy`, `drizzlepac`, and `photutils` for published research, please cite the
authors. Follow these links for more information about citing the libraries below:

* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `drizzlepac`](https://drizzlepac.readthedocs.io/en/latest/LICENSE.html)
* [Citing `photutils`](https://photutils.readthedocs.io/en/stable/license.html)

***
[Top of Page](#title)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 